In [9]:
# Displaying Information in Tabular Format
    # pip install tabulate

In [1]:
import requests 
from bs4 import BeautifulSoup

from dataclasses import dataclass

from tabulate import tabulate

@dataclass
class Stock:
    ticker: str
    exchange: str
    price: float=0
    currency: str='USD'
    usd_price: float=0

    def __post_init__(self):                                    
        price_info=get_price_info(self.ticker,self.exchange)    
        if price_info['ticker']==self.ticker:                   
            self.price=price_info['price']
            self.currency=price_info['currency']
            self.usd_price=price_info['usd_price']


@dataclass
class Position:
    stock: Stock
    quantity: int


@dataclass
class Portfolio:
    positions: list[Position]

    def get_total_value(self):      # self is automatically populated, because get_totoal_val is an instance method & all instance methods in python have 'self' as their first argument
        total_value=0
        for position in self.positions:
            total_value+=position.quantity*position.stock.usd_price
        return total_value



def get_fx_to_usd(currency):
    fx_url=f'https://www.google.com/finance/quote/{currency}-USD' 
    response=requests.get(fx_url)                                    
    soup=BeautifulSoup(response.content,"html.parser")            
    fx_rate=soup.find('div',attrs={'data-last-price':True})     
    fx=float(fx_rate['data-last-price'])
    return fx


def get_price_info(ticker,exchange):                                    
    url=f'https://www.google.com/finance/quote/{ticker}:{exchange}'     
    response=requests.get(url)                                       
    soup=BeautifulSoup(response.content,"html.parser")               
    price_div=soup.find('div',attrs={'data-last-price':True})        
    price=float(price_div['data-last-price'])                        
    currency=price_div['data-currency-code']                         
    
    usd_price=price
    if(currency!='USD'):                                             
        fx=get_fx_to_usd(currency)
        usd_price=round(price*fx,2)
    
    return {
        'ticker':ticker,
        'exchange':exchange,
        'price':price,
        'currency':currency,
        'usd_price':usd_price
    }

In [11]:
shop=Stock('SHOP','TSE')
tsla=Stock('TSLA','NASDAQ')
aapl=Stock('AAPL','NASDAQ')
amzn=Stock('AMZN','NASDAQ')

portfolio=Portfolio([Position(shop,10), Position(tsla,5), Position(aapl,7), Position(amzn,10)])

# portfolio 
# portfolio.get_total_value()

In [12]:
def display_portfolio_summary(portfolio):
    
    if not isinstance(portfolio,Portfolio):
        raise TypeError('Please provide an instance of Portfolio Type')
    
    portfolio_value=portfolio.get_total_value()

    position_data=[]
    for position in portfolio.positions:
        position_data.append([
            position.stock.ticker,
            position.stock.exchange,
            position.quantity,
            position.stock.usd_price,
            position.quantity*position.stock.usd_price,
            position.quantity*position.stock.usd_price/portfolio_value*100
        ])

    print(tabulate(position_data,
                   headers=['Ticker', 'Exchange', 'Quantity', 'Price', 'Market Value', '% Allocation'],
                   tablefmt='psql',
                   floatfmt='.2f'
                   ))
    print(f'Total Portfolio Value: ${portfolio_value:,.2f}.')


In [13]:
display_portfolio_summary(portfolio)

+----------+------------+------------+---------+----------------+----------------+
| Ticker   | Exchange   |   Quantity |   Price |   Market Value |   % Allocation |
|----------+------------+------------+---------+----------------+----------------|
| SHOP     | TSE        |         10 |   68.67 |         686.70 |          14.21 |
| TSLA     | NASDAQ     |          5 |  290.38 |        1451.90 |          30.05 |
| AAPL     | NASDAQ     |          7 |  193.99 |        1357.93 |          28.10 |
| AMZN     | NASDAQ     |         10 |  133.56 |        1335.60 |          27.64 |
+----------+------------+------------+---------+----------------+----------------+
Total Portfolio Value: $4,832.13.
